# Preprocessing Data

## Import Library

In [1]:
import os
import numpy as np
import pandas as pd
from sktime.datasets import load_from_tsfile_to_dataframe
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical


## Path

In [2]:
TRAIN_DIR = r"D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Klasifikasi_Colposcopy\Dataset\Colposcopy_TRAIN.ts"
TEST_DIR = r"D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Klasifikasi_Colposcopy\Dataset\Colposcopy_TEST.ts"

X_train, y_train = load_from_tsfile_to_dataframe(TRAIN_DIR)
X_test,  y_test  = load_from_tsfile_to_dataframe(TEST_DIR)

print("Train path :", TRAIN_DIR)
print("Val path   :", TEST_DIR)

Train path : D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Klasifikasi_Colposcopy\Dataset\Colposcopy_TRAIN.ts
Val path   : D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Klasifikasi_Colposcopy\Dataset\Colposcopy_TEST.ts


## Mendeteksi Missing Value

In [3]:
def check_missing(df):
    missing_per_series = df.applymap(lambda ts: np.isnan(ts).sum())
    total_missing = missing_per_series.sum().sum()
    return missing_per_series, total_missing

missing_train, total_missing_train = check_missing(X_train)
missing_test, total_missing_test = check_missing(X_test)

print("Total missing values in TRAIN :", total_missing_train)
print("Total missing values in TEST  :", total_missing_test)

print("\nMissing per instance (TRAIN):")
print(missing_train.sum(axis=1).head())

Total missing values in TRAIN : 0
Total missing values in TEST  : 0

Missing per instance (TRAIN):
0    0
1    0
2    0
3    0
4    0
dtype: int64


C:\Users\user\AppData\Local\Temp\ipykernel_18640\2666175583.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  missing_per_series = df.applymap(lambda ts: np.isnan(ts).sum())


tidak ada data kosong(missing value) di data train maupun test

## Normalisasi Data

In [4]:
def minmax_normalize_series(ts):
    """
    Min-Max normalization for a single time series
    """
    ts = np.array(ts, dtype=float)
    min_val = ts.min()
    max_val = ts.max()

    if max_val - min_val == 0:
        return np.zeros_like(ts)
    
    return (ts - min_val) / (max_val - min_val)


def normalize_dataframe(df):
    """
    Apply min-max normalization to all time series in a DataFrame
    """
    return df.applymap(minmax_normalize_series)



X_train_norm = normalize_dataframe(X_train)
X_test_norm  = normalize_dataframe(X_test)

print("Normalisasi selesai.")

Normalisasi selesai.


C:\Users\user\AppData\Local\Temp\ipykernel_18640\3514053908.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(minmax_normalize_series)


fit normalisasi hanya digunakan pada data train,data test hanya di transform

## Reshape Data Time series ke Format LSTM

In [5]:
def df_to_lstm_input(df):
    """
    Convert DataFrame (with time series in cells)
    into 3D numpy array for LSTM
    """
    X = np.stack(df.iloc[:, 0].values)
    
    X = X[..., np.newaxis]
    
    return X


X_train_lstm = df_to_lstm_input(X_train_norm)
X_test_lstm  = df_to_lstm_input(X_test_norm)

print("Shape X_train_lstm:", X_train_lstm.shape)
print("Shape X_test_lstm :", X_test_lstm.shape)

Shape X_train_lstm: (99, 180, 1)
Shape X_test_lstm : (101, 180, 1)


## Encoding class

In [6]:
num_classes = 6

y_train_enc = to_categorical(y_train, num_classes=num_classes)
y_test_enc  = to_categorical(y_test,  num_classes=num_classes)

print("Shape y_train_enc:", y_train_enc.shape)
print("Shape y_test_enc :", y_test_enc.shape)

Shape y_train_enc: (99, 6)
Shape y_test_enc : (101, 6)


In [7]:
import numpy as np

np.savez(
    "colposcopy_lstm_data.npz",
    X_train_lstm=X_train_lstm,
    y_train=y_train,              
    y_train_enc=y_train_enc,
    X_test_lstm=X_test_lstm,
    y_test=y_test,                
    y_test_enc=y_test_enc
)

print("Data preprocessing lengkap berhasil disimpan")


Data preprocessing lengkap berhasil disimpan
